In [1]:
import numpy as np
import sys
import scipy.stats
import re
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../codes')

In [4]:
%pprint

Pretty printing has been turned OFF


In [5]:
from VMSfunctions.Common import *
from VMSfunctions.Chemicals import *
from VMSfunctions.Chromatograms import *
from VMSfunctions.MassSpec import *
from VMSfunctions.Controller import *

# Mass Spec test

In [6]:
base_dir = 'C:\\Users\\joewa\\University of Glasgow\\Vinny Davies - CLDS Metabolomics Project\\'
base_dir = 'C:\\Users\\Vinny\\OneDrive - University of Glasgow\\CLDS Metabolomics Project\\'
cc = ChromatogramCreator(os.path.join(base_dir, 'Trained Models\\chromatogram_19_beers.csv.gz'))

In [7]:
hmdb = load_obj(os.path.join(base_dir, 'Trained Models\\hmdb_compounds.p'))

In [8]:
ps = load_obj(os.path.join(base_dir, 'Trained Models\\peak_sampler_beer1pos_fragmentation.p'))
chemicals = ChemicalCreator(ps)

# Improve Chemical Creator

Need to test the chemical creator in more situations to make sure the code works as intended

In [9]:
min_ms1_intensity = 2E5
rt_range = [(3*60, 21*60)]
mz_range = [(0, 1050)]
set_log_level_debug()
dataset = chemicals.sample(cc, mz_range, rt_range, min_ms1_intensity, 200, 2, "Known", None, hmdb, 0.1)

DEBUG:ChemicalCreator:200 ms1 peaks to be created.
DEBUG:ChemicalCreator:i = 0
DEBUG:ChemicalCreator:i = 25
DEBUG:ChemicalCreator:i = 50
DEBUG:ChemicalCreator:i = 75
DEBUG:ChemicalCreator:i = 100
DEBUG:ChemicalCreator:i = 125
DEBUG:ChemicalCreator:i = 150
DEBUG:ChemicalCreator:i = 175


In [ ]:
# min_ms1_intensity = 2E5
# min_rt = 3*60
# max_rt = 21*60
# chemicals = ChemicalCreator(ps)
# dataset = chemicals.sample_from_chromatograms(cc, min_rt, max_rt, min_ms1_intensity, ms_levels=2)

# TODO: Add way to specify n and then sample n chromatograms

# Multiple Datasets for DsDA

In [10]:
min_ms1_intensity = 2E5
rt_range = [(3*60, 21*60)]
mz_range = [(0, 1000)]
set_log_level_debug()
dataset = chemicals.sample(cc, mz_range, rt_range, min_ms1_intensity, 200, 2, "Known", None, hmdb, 0.1)

DEBUG:ChemicalCreator:200 ms1 peaks to be created.
DEBUG:ChemicalCreator:i = 0
DEBUG:ChemicalCreator:i = 25
DEBUG:ChemicalCreator:i = 50
DEBUG:ChemicalCreator:i = 75
DEBUG:ChemicalCreator:i = 100
DEBUG:ChemicalCreator:i = 125
DEBUG:ChemicalCreator:i = 150
DEBUG:ChemicalCreator:i = 175


In [11]:
n_samples = [10]
classes = ["class1"]
intensity_noise_sd = [1000]
change_probabilities = None
change_differences_means = None
change_differences_sds = None
dropout_probabilities = None 
experimental_classes = None 
experimental_probabilitities = None 
experimental_sds = None

multiple_samples = MultiSampleCreator(dataset, n_samples, classes, intensity_noise_sd, 
                                      change_probabilities, change_differences_means, change_differences_sds, dropout_probabilities,
                                     experimental_classes, experimental_probabilitities, experimental_sds)

DEBUG:MultiSampleCreator:Classes, Statuses and Differences defined.
DEBUG:MultiSampleCreator:Dataset 1 of 10 created.
DEBUG:MultiSampleCreator:Dataset 2 of 10 created.
DEBUG:MultiSampleCreator:Dataset 3 of 10 created.
DEBUG:MultiSampleCreator:Dataset 4 of 10 created.
DEBUG:MultiSampleCreator:Dataset 5 of 10 created.
DEBUG:MultiSampleCreator:Dataset 6 of 10 created.
DEBUG:MultiSampleCreator:Dataset 7 of 10 created.
DEBUG:MultiSampleCreator:Dataset 8 of 10 created.
DEBUG:MultiSampleCreator:Dataset 9 of 10 created.
DEBUG:MultiSampleCreator:Dataset 10 of 10 created.


In [12]:
print(multiple_samples.samples[0][0])
print(multiple_samples.samples[1][0])
print(multiple_samples.samples[2][0])

KnownChemical - 'C9H12NO5PS' rt=951.58 max_intensity=275420.62
KnownChemical - 'C9H12NO5PS' rt=951.58 max_intensity=275097.52
KnownChemical - 'C9H12NO5PS' rt=951.58 max_intensity=275157.88


# Create Multiple Datasets

In [13]:
min_ms1_intensity = 2E5
rt_range = [(3*60, 21*60)]
mz_range = [(0, 1000)]
set_log_level_debug()
dataset = chemicals.sample(cc, mz_range, rt_range, min_ms1_intensity, 200, 2, "Known", None, hmdb)

DEBUG:ChemicalCreator:200 ms1 peaks to be created.
DEBUG:ChemicalCreator:i = 0
DEBUG:ChemicalCreator:i = 25
DEBUG:ChemicalCreator:i = 50
DEBUG:ChemicalCreator:i = 75
DEBUG:ChemicalCreator:i = 100
DEBUG:ChemicalCreator:i = 125
DEBUG:ChemicalCreator:i = 150
DEBUG:ChemicalCreator:i = 175


In [14]:
n_samples = [5,5,5]
classes = ["class1","class2","class3"]
intensity_noise_sd = [1000]
change_probabilities = [0.2,0.2]
change_differences_means = [10000,20000]
change_differences_sds = [0,0] # 0 means fixed difference between class and class1
dropout_probabilities = [0.2,0.2,0.2]
experimental_classes = None # [["male","female"],["Positive","Negative","Unknown"]]
experimental_probabilitities = None # [[0.5,0.5],[0.33,0.33,0.34]]
experimental_sds = None # [[250],[250]]

multiple_samples = MultiSampleCreator(dataset, n_samples, classes, intensity_noise_sd, 
                                      change_probabilities, change_differences_means, change_differences_sds, dropout_probabilities,
                                     experimental_classes, experimental_probabilitities, experimental_sds)

DEBUG:MultiSampleCreator:Classes, Statuses and Differences defined.
DEBUG:MultiSampleCreator:Dataset 1 of 15 created.
DEBUG:MultiSampleCreator:Dataset 2 of 15 created.
DEBUG:MultiSampleCreator:Dataset 3 of 15 created.
DEBUG:MultiSampleCreator:Dataset 4 of 15 created.
DEBUG:MultiSampleCreator:Dataset 5 of 15 created.
DEBUG:MultiSampleCreator:Dataset 6 of 15 created.
DEBUG:MultiSampleCreator:Dataset 7 of 15 created.
DEBUG:MultiSampleCreator:Dataset 8 of 15 created.
DEBUG:MultiSampleCreator:Dataset 9 of 15 created.
DEBUG:MultiSampleCreator:Dataset 10 of 15 created.
DEBUG:MultiSampleCreator:Dataset 11 of 15 created.
DEBUG:MultiSampleCreator:Dataset 12 of 15 created.
DEBUG:MultiSampleCreator:Dataset 13 of 15 created.
DEBUG:MultiSampleCreator:Dataset 14 of 15 created.
DEBUG:MultiSampleCreator:Dataset 15 of 15 created.


In [15]:
print(multiple_samples.chemical_statuses[0][0:3])
print(multiple_samples.chemical_statuses[1][0:3])
print(multiple_samples.chemical_statuses[2][0:3])

print( )

print(multiple_samples.samples[0][0:3])
print(multiple_samples.samples[5][0:3])
print(multiple_samples.samples[10][0:3])


['unchanged' 'missing' 'missing']
['unchanged' 'unchanged' 'unchanged']
['unchanged' 'unchanged' 'unchanged']

[KnownChemical - 'C3H5NO3' rt=858.89 max_intensity=1584634.58, KnownChemical - 'C15H23NO3' rt=1042.34 max_intensity=1975856.83, KnownChemical - 'C7H3Cl5O' rt=295.82 max_intensity=1043712.77]
[KnownChemical - 'C3H5NO3' rt=858.89 max_intensity=1585713.75, KnownChemical - 'C10H14N5O10PS' rt=1195.91 max_intensity=2223211.99, KnownChemical - 'C12H18' rt=1100.04 max_intensity=473964.96]
[KnownChemical - 'C3H5NO3' rt=858.89 max_intensity=1585603.54, KnownChemical - 'C10H14N5O10PS' rt=1195.91 max_intensity=2222614.86, KnownChemical - 'C12H18' rt=1100.04 max_intensity=473417.23]


In [16]:
i = 0
for j in range(len(multiple_samples.samples)):
    if j>0 and multiple_samples.sample_classes[j] != multiple_samples.sample_classes[j-1]:
        print( )
    if multiple_samples.sample_classes[j] == multiple_samples.classes[0]:
        print(multiple_samples.samples[j][i].max_intensity, multiple_samples.sample_classes[j])
    else:
        print(multiple_samples.samples[j][i].max_intensity, multiple_samples.sample_classes[j],
              multiple_samples.chemical_statuses[np.where(np.array(multiple_samples.classes)==multiple_samples.sample_classes[j])[0][0]-1][i],
             multiple_samples.chemical_differences_from_class1[np.where(np.array(multiple_samples.classes)==multiple_samples.sample_classes[j])[0][0]][i])

1584634.5780402194 class1
1585592.7249060061 class1
1582887.8159598918 class1
1586173.9940251221 class1
1584756.9602212952 class1

1585713.750675501 class2 unchanged 0
1584574.0368094775 class2 unchanged 0
1586536.6473650574 class2 unchanged 0
1585777.0372175665 class2 unchanged 0
1583404.1813317295 class2 unchanged 0

1585603.5365958293 class3 unchanged 0
1584807.2282735447 class3 unchanged 0
1583747.8860192234 class3 unchanged 0
1584835.5708120842 class3 unchanged 0
1584800.0236169572 class3 unchanged 0


In [17]:
max_rt = 10*60                  # the maximum retention time of scans to generate
N = 1                           # top-5 DDA fragmentation
isolation_window = 0.5          # the isolation window in Dalton around a selected precursor ion
rt_tol = 15                     # the rt window around a selected precursor ion to prevent it from fragmented multiple times
min_ms1_intensity = 2E5         # the minimum ms1 peak intensity
multi_dataset = multiple_samples.samples[0]

mass_spec = IndependentMassSpectrometer(POSITIVE, multi_dataset, density=ps.density_estimator)
controller = TopNController(mass_spec, N, isolation_window, rt_tol, min_ms1_intensity)

set_log_level_warning() # We don't want to see too many messages as the controller is running
# set_log_level_info()
# set_log_level_debug()

controller.run(0, max_rt)

600.006497110021it [00:03, 177.37it/s]                                                                                 


In [18]:
# controller.write_mzML('my_analysis2', '../models/out2.mzML')
# save_obj(controller, '../models/noisy_top_N_controllers.p')

# Example of DIA Methods across an Entire Dataset

Generate test dataset

In [19]:
min_ms1_intensity = 2E5
rt_range = [(180, 1260)]
mz_range = [(0, 1050)]
chemicals = ChemicalCreator(ps)
dataset = chemicals.sample(cc, mz_range, rt_range, min_ms1_intensity, 3000, 2, "Unknown", None, None, fixed_mz = True)

Kaufmann and Walker Nested DIA method with Even windows (10)

In [20]:
max_rt = 10*60    
dia_design = "kaufmann"
window_type = "even"
kaufmann_design = "nested"
extra_bins = 0
num_windows=None # num_windows fixed for kaufmann methods, so not required

mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, density=ps.density_estimator)

In [29]:
print(dataset[0].rt)
print(dataset[0].chromatogram.rts)
print(dataset[0].chromatogram.raw_rts)

1240.3724806753685
[0.    1.274 2.514 3.753 4.992 6.27  7.51  8.749]
[636.336 637.61  638.85  640.089 641.328 642.606 643.846 645.085]


In [28]:
query_rt = range(1237,1255,1)
for i in query_rt:
    print(i)
    print(mass_spec._get_all_mz_peaks(dataset[0], i, 1, [[(0,2000)]]))

1237
None
1238
None
1239
None
1240
None
1241
[(204.9815958453365, 149921.9555794806)]
1242
[(204.9815958453365, 199005.2912506645)]
1243
[(204.9815958453365, 122273.03058442891)]
1244
[(204.9815958453365, 293409.76542752876)]
1245
[(204.9815958453365, 387683.65565441933)]
1246
[(204.9815958453365, 350036.91713870637)]
1247
[(204.9815958453365, 231295.42644278373)]
1248
[(204.9815958453365, 111632.02792270221)]
1249
[(204.9815958453365, 143757.96840745801)]
1250
None
1251
None
1252
None
1253
None
1254
None


In [26]:
max_rt = 10*60    
dia_design = "kaufmann"
window_type = "even"
kaufmann_design = "nested"
extra_bins = 0
num_windows=None # num_windows fixed for kaufmann methods, so not required

mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, density=ps.density_estimator)
controller = TreeController(mass_spec, dia_design, window_type, kaufmann_design, extra_bins, num_windows)

set_log_level_warning()

controller.run(0, rt_range[0][1])

1260.087895559426it [01:36, 13.03it/s]                                                                                 


Standard DIA with Even Windows

In [16]:
max_rt = 10*60    
dia_design = "basic"
window_type = "even"
kaufmann_design = None
extra_bins = 0 # cant have extra bins
num_windows=10

mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, density=ps.density_estimator)
controller = TreeController(mass_spec, dia_design, window_type, kaufmann_design, extra_bins, num_windows)

set_log_level_warning()

controller.run(0, rt_range[0][1])

1260.016190138303it [00:57, 21.81it/s]                                                                                 


Kaufmann and Walker Nested DIA method with Percentile-Based windows

In [27]:
max_rt = 10*60    
dia_design = "kaufmann"
window_type = "percentile"
kaufmann_design = "nested"
extra_bins = 0
num_windows= None # num_windows fixed for kaufmann methods, so not required

mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, density=ps.density_estimator)
controller = TreeController(mass_spec, dia_design, window_type, kaufmann_design, extra_bins, num_windows)

set_log_level_warning()

controller.run(0, rt_range[0][1])

1260.0673680589555it [01:37, 12.90it/s]                                                                                


Standard DIA with Percentile-Based Windows (10)

In [18]:
max_rt = 10*60    
dia_design = "basic"
window_type = "percentile"
kaufmann_design = None
extra_bins = 0 # cant have extra bins
num_windows=10

mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, density=ps.density_estimator)
controller = TreeController(mass_spec, dia_design, window_type, kaufmann_design, extra_bins, num_windows)

set_log_level_warning()

controller.run(0, rt_range[0][1])

1260.049054431451it [00:56, 22.14it/s]                                                                                 


# Evaluate of DIA results on Restricted Dataset

Create dataset in restricted RT space

In [9]:
N = 100
prop_ms2_mass = 0.7
dataset = []
chrom = FunctionalChromatogram("uniform",[0,2000])
for i in range(N):
    mz = ps.sample(1,1)[0].mz
    chem = UnknownChemical(mz, 0, 1E5, chrom, children=None)
    n_children = int(ps.density_estimator.n_peaks(2, 1))
    parent_mass_prop = [1/n_children for k in range(n_children)]
    children = []
    for j in range(n_children):
        mz = ps.sample(2, 1)[0].mz
        children.append(MSN(mz, 2, prop_ms2_mass, parent_mass_prop[j], None, chem))
    chem.children = children
    dataset.append(chem)

In [82]:
scans_even_kauf = DiaRestrictedScanner(dataset, ps, 200, [(0,1000)], "kaufmann", "even", "nested", 0, None)
scans_percent_kauf = DiaRestrictedScanner(dataset, ps, 200, [(0,1000)], "kaufmann", "percentile", "nested", 0, None)
num_windows = 10
scans_even_basic = DiaRestrictedScanner(dataset, ps, 200, [(0,1000)], "basic", "even", None, 0, num_windows)
scans_percent_basic = DiaRestrictedScanner(dataset, ps, 200, [(0,1000)], "basic", "percentile", None, 0, num_windows)

In [83]:
scans_even_kauf.ms2_intensities

[[0.32434436104160874, 1.309390198279087, 2.3569023569023564, 0.8417508417508417, 2.07961972667855, 0.9303561935140882, 0.8221752407798918, 0.47775047775047774, 1.8607123870281763, 1.1047979797979797, 3.928170594837261, 0.6670478368591576], [1.6069788797061526, 1.4141414141414141, 0.5992124636192432, 2.7195027195027195, 1.7676767676767677, 0.8221752407798918, 2.2095959595959593, 2.3569023569023564, 1.473063973063973, 1.4141414141414141, 1.4141414141414141, 0.4591368227731864, 1.1047979797979797, 0.8838383838383839, 1.1784511784511782, 17.676767676767675, 4.419191919191919, 0.7365319865319865, 1.1784511784511782, 0.8034894398530763, 1.473063973063973, 1.309390198279087, 2.7195027195027195, 1.3597513597513597, 1.7676767676767677, 0.6546950991395435, 1.6069788797061526, 0.9555009555009555, 1.1784511784511782, 1.2190874259839777, 0.5123700775874689, 3.5353535353535355, 1.071319253137435, 1.0101010101010102, 0.8034894398530763, 1.7676767676767677, 0.6932065755595167, 0.7685551163812033, 1.3

In [81]:
class DiaRestrictedScanner(object):
    def __init__(self, dataset, density_estimator, initial_time, ms1_range, dia_design, window_type, kaufmann_design, extra_bins, num_windows=None):
        self.dataset = dataset
        self.density_estimator = density_estimator
        self.initial_time = initial_time
        self.ms1_range = ms1_range
        self.dia_design = dia_design
        self.window_type = window_type
        self.kaufmann_design = kaufmann_design
        self.extra_bins = extra_bins
        self.num_windows = num_windows
        
        self.mass_spec = IndependentMassSpectrometer(POSITIVE, self.dataset, density=self.density_estimator)
        self.ms1_scan_mzs = self._ms1_scan()
        self.locations = self._calculate_windows()
        self.ms2_scans = self._ms2_scans()
        self.ms2_mzs, self.ms2_intensities = self._reformat_scans()
        
    def _calculate_windows(self):
        return DiaWindows(self.ms1_scan_mzs, self.ms1_range, self.dia_design, self.window_type, self.kaufmann_design, self.extra_bins, self.num_windows).locations
    
    def _ms1_scan(self):
        ms1_scan = [mass_spec._get_all_mz_peaks(dataset[i], 100, 1, [[(0,1E5)]])[0] for i in range(len(dataset))]
        return ([ms1_scan[i][0] for i in range(len(ms1_scan))])
    
    def _ms2_scans(self):
        ms2_scans = [[] for i in range(len(self.locations))]
        for window_index in range(len(self.locations)):
            locations = self.locations[window_index]
            for dataset_index in range(len(self.dataset)):
                new_ms2s = self.mass_spec._get_all_mz_peaks(self.dataset[dataset_index], 101 + window_index, 2, locations)
                if new_ms2s is not None:
                    ms2_scans[window_index].append(new_ms2s[0])
        return ms2_scans
    
    def _reformat_scans(self):
        ms2_mzs = [[] for i in self.ms2_scans]
        ms2_intensities = [[] for i in self.ms2_scans]
        for i in range(len(self.ms2_scans)):
            for j in range(len(self.ms2_scans[i])):
                ms2_mzs[i].append(self.ms2_scans[i][j][0])
                ms2_intensities[i].append(self.ms2_scans[i][j][1])        
        return ms2_mzs, ms2_intensities

In [ ]:
class DiaRestrictedAnalyser(object)
    def __init__(self, DiaRestrictedAnalyser):
        
# can probably just use old functions now - rather than this

# Calculate DIA performance (Overall)

In [ ]:
min_ms1_intensity = 2E5
rt_range = [(180, 1260)]
mz_range = [(0, 1050)]
chemicals = ChemicalCreator(ps)
dataset = chemicals.sample(cc, mz_range, rt_range, min_ms1_intensity, 50, 2, "Unknown", None, None, fixed_mz = True)
big_dataset = chemicals.sample(cc, mz_range, rt_range, min_ms1_intensity, 3000, 2, "Unknown", None, None, fixed_mz = True)

In [ ]:
max_rt = 10*60    
dia_design = "basic"
window_type = "even"
kaufmann_design = None
extra_bins = 0
num_windows=5

mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, density=ps.density_estimator)
controller = TreeController(mass_spec, dia_design, window_type, kaufmann_design, extra_bins, num_windows)

set_log_level_warning() # We don't want to see too many messages as the controller is running
# set_log_level_info()
# set_log_level_debug()

controller.run(0, rt_range[0][1])

In [ ]:
PROTON_MASS = 1.00727645199076

ca = Controller_Analyser(controller)
print(ca.proportion_ms1_fragmented)


mz2 = ca.list_of_ms2_mzs_dataset[0] + PROTON_MASS

times_alluting = []
for scan in controller.scans[2]:
    if mz2 in scan.mzs:
        times_alluting.append(scan.rt)
times_alluting # we know that this


for each ms2
    loop over each ms1
        loop over scans
            does it match the allution status of the ms2
        
- base on logic that we know the exact ellution times of the chemical
- and no noise in mzs

In [ ]:
number_of_possible_chemicals = []
counter = 0
for mz_dataset in ca.list_of_ms2_mzs_dataset[0:2]:
    counter += 1
    print(counter)
    if mz_dataset + PROTON_MASS not in ca.list_of_ms2_mzs_controller:
        possible_chemicals = len(controller.mass_spec.chemicals) 
        # this isnt technically accurate
        # should technically be the number of chemicals we arent sure whether have been fragmented
        number_of_possible_chemicals.append(possible_chemicals)
    else:
        possible_chemicals = 0
        allution_times = []
        for scan in controller.scans[2]:
            if mz2 in scan.mzs:
                allution_times.append(scan.rt)
        allution_times = np.array(allution_times)
        print(allution_times)
        for chem in controller.mass_spec.chemicals:
            print(chem.rt)
            chemical_rt_range = [chem.rt, chem.rt + chem.chromatogram.rts[-1]]
            if np.logical_and(chemical_rt_range[0] < allution_times,allution_times < chemical_rt_range[1]).all():
                print("here")
                scan_num = 0
                allution_status_matches = True
                while allution_status_matches and scan_num < len(controller.scans[2]):
                    # check status
                    ms2_elluting = (mz_dataset + PROTON_MASS) in controller.scans[2][scan_num].mzs
                    ms2_scan_status = controller.mass_spec._get_mz_peaks(chem, controller.scans[2][scan_num].rt, 1, controller.scans[2][scan_num].isolation_windows, 1, 1)
                    ms2_in_scan = (ms2_scan_status is not None)
                    if ms2_elluting != ms2_in_scan:
                        allution_status = False
                    scan_num += 1
                possible_chemicals += int(allution_status)
        number_of_possible_chemicals.append(possible_chemicals)

In [ ]:
number_of_possible_chemicals

In [ ]:
int(False)

In [ ]:
# only works when MS1 dont have shared MS2s
# only works when no isotopes or adducts
class Controller_Analyser(object):
    def __init__ (self, controller):
        
        # calculate observed MS2 mzs
        self.list_of_ms2_mzs_controller = []
        for scan in controller.scans[2]:
            self.list_of_ms2_mzs_controller.extend(scan.mzs.tolist())
        self.list_of_ms2_mzs_controller = np.unique(np.array(self.list_of_ms2_mzs_controller))
        
        # calculate list of all MS2 mzs
        self.list_of_ms2_mzs_dataset = []
        for chem in controller.mass_spec.chemicals:
            for child in chem.children:
                self.list_of_ms2_mzs_dataset.append(child.isotopes[0][0])
            
        # calculate observed MS1 mzs
        self.list_of_ms1_mzs_controller = []
        for scan in controller.scans[1]:
            self.list_of_ms1_mzs_controller.extend(scan.mzs.tolist())
        self.list_of_ms1_mzs_controller = np.unique(np.array(self.list_of_ms1_mzs_controller))
        
        # proportion of MS1 observed
        self.proportion_ms1_fragmented = len(self.list_of_ms1_mzs_controller) / len(controller.mass_spec.chemicals)
        
        # calculate all ms2 RTs
        self.ms2_rts = []
        for scan in controller.scans[2]:
            self.ms2_rts.append(scan.rt)
        
        # calculate rt and mz region for each ms2 mz from dataset
        

Percentage of MS2 Fragmented

In [ ]:
max_rt = 10*60    
dia_design = "basic"
window_type = "even"
kaufmann_design = None
extra_bins = 0
set_log_level_warning() # We don't want to see too many messages as the controller is running
# set_log_level_info()
# set_log_level_debug()
mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, density=ps.density_estimator)

percent_id = []
for i in range(num_windows):
    num_windows=range(2,30,2)
    controller = TreeController(mass_spec, dia_design, window_type, kaufmann_design, extra_bins, num_windows[i])
    controller.run(0, max_rt)
    CA = Controller_Analyser(controller)
    percent_id.append(CA.percentage_ms2_fragmented)
    

- can scan the MS at RT of MS2 scan, gets list of MS1 peaks in RT range (already have list of MS2 peaks). Will then need scan_location to get the MS1s in MZ range (needs adding)

Assume we know which MS1 the MS2 come from

# To Do

- Convolutional neural networks to identify ROI
    - pad so all same length
    - link ROI to final peaks to see how many peaks there are
    - predict how many peaks in a ROI

- Talk to Stefan about how basic strategies in the MS would work
    - how does interact with exclusion window
    - what does it do if we see no peaks